In [1]:
import json
import os
import time

from dag_nao.modules.readers.read_files import MultipleFilesReader
from dag_nao.modules.llm_backend.llm_backend import LLMBackend
from dag_nao.modules.common.json_to_excel import ExcelWriter

In [32]:
# Gestion de paths
num_dossier = "2022DOS1246222"
entreprise = "consort"

output_path = f"/workspace/outputs/{num_dossier}"
os.makedirs(output_path, exist_ok=True)
os.makedirs(os.path.join(output_path, "json"), exist_ok=True)

rec_pdf = "/workspace/data/03-AppelsOffre-anonyme/2022DOS1246222 - CDP Telecoms/ao_sncf/REC-CDP_Telecoms.pdf"
cdc_pdf = "/workspace/data/03-AppelsOffre-anonyme/2022DOS1246222 - CDP Telecoms/ao_sncf/CDC-CDP_Telecoms.pdf"
cv_pdf = "/workspace/data/03-AppelsOffre-anonyme/2022DOS1246222 - CDP Telecoms/offres_techniques/TT_CONSULTING-1328242/CV-AMO.pdf"
bpu = "/workspace/data/03-AppelsOffre-anonyme/2022DOS1246222 - CDP Telecoms/offres_financieres/TT_CONSULTING-1328242/BPU.xlsx"

# PARSING DES DOCUMENTS

In [27]:
# Instanciate reader
extractor = MultipleFilesReader()

files_path = [[rec_pdf, cdc_pdf], [cv_pdf, bpu]]

# Extract information from files specified in the toml
all_info = extractor.read_all_documents(files_path)

In [20]:
all_info[0][rec_pdf]

"DEBUT DE LA PAGE 1\n\n\n \nREGLEMENT DE CONSULTATION PRESTATIONS INTELLECTUELLES AU SEIN DU REFERENCEMENT CONTRAT-CADRE AST & DIGITAL.  \nDOSSIER DE CONSULTATION N°2022DOS1246222 \nAcheteur (société pilotant la procédure de consultation) : SNCF, 2 place aux Etoiles - 93200 Saint-Denis, \nImmatriculation RCS : RCS Bobigny n°552 049 447  \nDate limite de réception des offres1 : 18/01/2023 à 12h00 \nAdresse URL du profil d’acheteur (e@si) :   \nhttps://sncf.bravosolution.com/web/login.html \nDate limite de remise des questions 2 : 09/01/2023 à 12h00 \nBrief technique3 : ☐ Oui ☒ Non \nOffre technique : \n- \nMémoire technique \n- \nCV des intervenants affectés à la réalisation de la \nprestation \n- \nPlanning ressourcé (nombre de jours prévus par profil) \n- \nTout autre document utile à la bonne compréhension \nde l’offre.  \n \nL’offre technique ne devra pas dépasser 30 pages. Aucune \nmention de prix ne doit apparaître dans l’offre technique. \nOffre financière :  \nLe soumissionnaire

In [24]:
print(all_info[1][cv_pdf])

DEBUT DE LA PAGE 1


Expérience - 15 ans
Faits marquants
Connaissance du contexte SNCF (Franck Ferreira Da Costa)
Connaissance de la Vidéo Protection
Chef de projet de bout en bout (recueil de besoin, rédaction de
cahier des charges…)
Formation Power BI en cours – e-learning
FORMATION
2022 Certification ITIL Foundation V4
2019 Certification ITIL Foundation V3
2014 Formation « Les clés de la réussite dans le management de
projet » Centre Learning Tree International– Paris (75)
2009 Diplôme de responsable de gestion - option Marketing (Bac
+5) – ESM-A - Ecole Supérieure de Management en Alternance de
Marne la Vallée(77).
2005 Licence d’informatique – Université de Nice – Sophia Antipolis
(06)
2004 DUT d’informatique – option imagerie numérique – IUT de
Nice - Sophia Antipolis (06)
COMPETENCES TECHNIQUES



FIN DE LA PAGE 1
DEBUT DE LA PAGE 2


Bureautique / logiciels Word, Excel, PowerPoint, Service Now,
Centreon.
Langages C, C++, Java, Html, XML, PHP, SQL.
Systèmes UNIX, MICROSOFT
COMPE

# STRUCTURATION DE L'INFORMATION AVEC CLAUDE

In [28]:
# Instanciate LLMBackend
llm_backend = LLMBackend()

In [29]:
def save_dict(d: dict, path: str) -> None:
    # Serializing json
    json_object = json.dumps(d, indent=4)

    with open(path, "w") as outfile:
        outfile.write(json_object)

## AO

In [17]:
metadata = True
os.makedirs(os.path.join(output_path, "json", "ao"), exist_ok=True)


# CDC
cdc_info = all_info[0][cdc_pdf]
output_cdc = os.path.join(output_path, "json", "ao", "formatted_json_cdc.json")

semantic_type_document = "CDC"
(
    formatted_json_cdc,
    list_jsons_similarity_distance,
) = llm_backend.structure_file_from_template(
    file_content=cdc_info,
    semantic_type_document=semantic_type_document,
    retries=4,
    add_metadata=metadata,
)
time.sleep(5)
save_dict(d=formatted_json_cdc, path=output_cdc)
time.sleep(30)


# REC
rec_info = all_info[0][rec_pdf]
output_rec = os.path.join(output_path, "json", "ao", "formatted_json_rec.json")

semantic_type_document = "REC"
(
    formatted_json_rec,
    list_jsons_similarity_distance,
) = llm_backend.structure_file_from_template(
    file_content=rec_info,
    semantic_type_document=semantic_type_document,
    retries=3,
    add_metadata=metadata,
)
time.sleep(5)
save_dict(d=formatted_json_rec, path=output_rec)
time.sleep(30)

Returned JSON is valid and types are correct.
Parsed Response: objectif_de_la_mission="Participer à l'administration, l'exploitation, la sécurisation et le maintien en condition opérationnelle de la plateforme Data de SNCF Gares & Connexions hébergée sur Microsoft Azure" duree_de_la_mission=365 nombre_de_profils=1 types_de_profils=[TypeProfile(type='914.S.PRO'), TypeProfile(type='914.E.PRO (variante expérimenté)')] taches_et_responsabilites=[Tache(tache_et_responsabilite="Participer à l'administration, l'exploitation, la sécurisation et le maintien en condition opérationnelle de la plateforme Data"), Tache(tache_et_responsabilite="Mettre en œuvre de nouvelles solutions d'infrastructures techniques Microsoft Azure"), Tache(tache_et_responsabilite='Fournir les outils et ressources Microsoft Azure nécessaires aux projets'), Tache(tache_et_responsabilite="Faire évoluer les ressources existantes dans une démarche d'amélioration continue"), Tache(tache_et_responsabilite='Participer à la réso

## CV

In [30]:
## CV
metadata = True

# get info
tech_info = all_info[1][cv_pdf]

# output path
output_cv = os.path.join(
    output_path, "json", entreprise, "formatted_json_cv.json"
)
os.makedirs(os.path.join(output_path, "json", entreprise), exist_ok=True)

semantic_type_document = "CV"
(
    formatted_json_cv,
    list_jsons_similarity_distance,
) = llm_backend.structure_file_from_template(
    file_content=tech_info,
    semantic_type_document=semantic_type_document,
    retries=7,
    add_metadata=metadata,
)
time.sleep(5)
save_dict(d=formatted_json_cv, path=output_cv)

Error: 1 validation error for CVExperiencesJsonOutputFormat
experiences_entreprise -> 0 -> date_fin_experience
  invalid date format (type=value_error.date)
Error: 1 validation error for CVExperiencesJsonOutputFormatMetadata
experiences_entreprise -> 0 -> date_fin_experience -> 0
  invalid date format (type=value_error.date)
{'score': 0.09727626459143969}
{'score': 0.09727626459143969}
{'score': 0.09727626459143969}
{'score': 0.09727626459143969}
{'score': 0.09727626459143969}
{'score': 0.09727626459143969}
{'score': 0.09727626459143969}
Returned JSON is valid and types are correct.
Parsed Response: experiences_entreprise=[EntrepriseExperienceTaches(nom_entreprise='SNCF', nom_poste='Chef de projet Monitoring IP', role=[TacheEntreprise(tache="Cataloguer et analyser les besoins en monitoring de performances pour les flux d'applications de signalisation ferroviaire (SF)"), TacheEntreprise(tache="Analyser l'intérêt et la mise en œuvre des outils de monitoring de performances des applicatio

In [15]:
save_dict(d=formatted_json_cv, path=output_cv)

## BPU

In [31]:
bpu_info = all_info[1][bpu]

# output paths
output_bpu = os.path.join(
    output_path, "json", entreprise, "formatted_json_bpu.json"
)
os.makedirs(os.path.join(output_path, "json", entreprise), exist_ok=True)

with open(output_bpu, "w") as outfile:
    json.dump(bpu_info, outfile)

# GENERATE EXCELs FROM JSONs

In [33]:
metadata = True

# output excel path, delete existing excel file
output_excel = f"/workspace/outputs/{num_dossier}/{num_dossier}.xlsx"
# os.remove(output_excel)
# instanciate ExcelWrite
W = ExcelWriter(output_excel, metadata)

# json files
json_path = os.path.join(output_path, "json")

for dir in os.listdir(json_path):
    abs_dir_path = os.path.join(json_path, dir)

    # check if directory
    if os.path.isdir(abs_dir_path):

        # APPEL D'OFFRE
        if "ao" in dir:
            json_files_ao = os.listdir(abs_dir_path)

            # check if directory is not empty
            if json_files_ao:
                # create worksheet
                W.new_worksheet("Appel d'offre")
                for file in json_files_ao:
                    if "cdc" in file:
                        # Write CDC info in excel sheet
                        W.json_to_excel_single_sheet(
                            os.path.join(abs_dir_path, file)
                        )
                    if "rec" in file:
                        # Write REC info in excel sheet
                        W.json_to_excel_single_sheet(
                            os.path.join(abs_dir_path, file)
                        )

        # REPONSE APPEL D'OFFRE
        else:
            json_files_reponse = os.listdir(abs_dir_path)

            # check if directory is not empty
            if json_files_reponse:
                # create a worksheet per company
                W.new_worksheet(f"{dir}")
                for file in json_files_reponse:
                    if "bpu" in file:
                        # Write BPU info in excel sheet
                        W.bpu_to_excel(os.path.join(abs_dir_path, file))
                    if "cv" in file:
                        # Write CV info in excel sheet
                        W.cv_to_excel(os.path.join(abs_dir_path, file))

W.workbook.close()

Bac+5
9
2017-01-01
9
Diplôme d'Ingénieur Institut Mine Télécoms spécialisé en réseaux et sécurité
9
LIST OF DICTS :
[{'tache': ['N/A', -1]}]
N/A
-1
ANGLAIS
9
INTERMEDIAIRE B2
9
SNCF Réseau
5
1 année, 6 mois
5
2021-04-01
5
2022-09-30
5
Chef de projet IT
5
LIST OF DICTS :
[{'tache': ['Gestion de Projets IT', 5]}, {'tache': ['Architecture WAN-MAN-LAN-TOIP', 5]}, {'tache': ['Déploiement FIBRE', 5]}, {'tache': ['Pilotage équipes Techniques', 5]}]
Gestion de Projets IT
5
Architecture WAN-MAN-LAN-TOIP
5
Déploiement FIBRE
5
Pilotage équipes Techniques
5
SFR Business
6
1 année, 8 mois
6
2019-01-01
6
2020-08-31
6
Chef de projets Télécoms B2B
6
LIST OF DICTS :
[{'tache': ['Architecture WAN-MAN-Téléphonie', 6]}, {'tache': ['Gestion de Projets Télécoms', 6]}, {'tache': ['Déploiement solutions télécoms B2B', 6]}, {'tache': ['Pilotage Sous traitants', 6]}, {'tache': ['Réseaux 4G', 6]}]
Architecture WAN-MAN-Téléphonie
6
Gestion de Projets Télécoms
6
Déploiement solutions télécoms B2B
6
Pilotage Sous t